In [1]:
import pandas as pd

In [2]:
#read file
file_content = pd.read_csv("C:/Users/SECTIO/Downloads/spam_ham_dataset/spam_ham_dataset.csv")

In [3]:
file_content

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [4]:
file_content['text'][1]

'Subject: hpl nom for january 9 , 2001\r\n( see attached file : hplnol 09 . xls )\r\n- hplnol 09 . xls'

In [5]:
#Import library
from nltk.corpus import stopwords
from nltk import *
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [6]:
# Remove stop words
stop = stopwords.words('english')
file_content['text'] = file_content['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [7]:
# Delete unwanted columns
Email_Data = file_content[['label', 'text']]

In [8]:
# Rename column names
Email_Data = Email_Data.rename(columns={"label":"Target", "text":"Email"})
Email_Data.head()

,Target,Email
0,ham,Subject: enron methanol ; meter # : 988291 fol...
1,ham,"Subject: hpl nom january 9 , 2001 ( see attach..."
2,ham,"Subject: neon retreat ho ho ho , ' around wond..."
3,spam,"Subject: photoshop , windows , office . cheap ..."
4,ham,Subject: : indian springs deal book teco pvr r...


In [9]:
#Delete Punctuation
Email_Data['Email'] = Email_Data['Email'].apply(lambda x: re.sub('[!@#$:).;,?&]','', x.lower()))
Email_Data['Email'] = Email_Data['Email'].apply(lambda x: re.sub(' ', ' ', x))
Email_Data['Email'].head(5)

0    subject enron methanol  meter   988291 follow ...
1    subject hpl nom january 9  2001 ( see attached...
2    subject neon retreat ho ho ho  ' around wonder...
3    subject photoshop  windows  office  cheap  mai...
4    subject  indian springs deal book teco pvr rev...
Name: Email, dtype: object

In [10]:
list_sentences_rawdata = Email_Data["Email"].fillna("_na_").values
list_classes = ["Target"]
target = Email_Data[list_classes].values
To_Process=Email_Data[['Email', 'Target']]

In [11]:
#Train and test split with 80:20 ratio
train, test = train_test_split(To_Process, test_size=0.2)
# Define the sequence lengths, max number of words and embedding dimensions
# Sequence length of each sentence. If more, truncate. If less, pad with zeros
MAX_SEQUENCE_LENGTH = 300

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
# Top 20000 frequently occurring words
MAX_NB_WORDS = 20000
# Get the frequently occurring words
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train.Email)
train_sequences = tokenizer.texts_to_sequences(train.Email)
test_sequences = tokenizer.texts_to_sequences(test.Email)
# dictionary containing words and their index
word_index = tokenizer.word_index
# print(tokenizer.word_index)
# total words in the corpus
print('Found %s unique tokens.' % len(word_index))

Found 43960 unique tokens.


In [13]:
# get only the top frequent words on train
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
# get only the top frequent words on test
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(train_data.shape)
print(test_data.shape)

(4136, 300)
(1035, 300)


In [14]:
train_labels = train['Target']
test_labels = test['Target'] 

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
# converts the character array to numeric array. Assigns levels to unique labels.
le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)

In [20]:
import numpy as np
print(le.classes_)
print(np.unique(train_labels, return_counts=True))
print(np.unique(test_labels, return_counts=True))

['ham' 'spam']
(array([0, 1]), array([2942, 1194], dtype=int64))
(array([0, 1]), array([730, 305], dtype=int64))


In [23]:
from tensorflow.keras.utils import to_categorical
labels_train = to_categorical(np.asarray(train_labels))
labels_test = to_categorical(np.asarray(test_labels))
print('Shape of data tensor:', train_data.shape)
print('Shape of label tensor:', labels_train.shape)
print('Shape of label tensor:', labels_test.shape)

Shape of data tensor: (4136, 300)
Shape of label tensor: (4136, 2)
Shape of label tensor: (1035, 2)


In [24]:
EMBEDDING_DIM = 100
print(MAX_SEQUENCE_LENGTH)

300


In [25]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D, SimpleRNN
from keras.models import Model
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

In [26]:
print('Training CNN 1D model.')

Training CNN 1D model.


In [27]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [28]:
model.fit(train_data, labels_train, batch_size=64, epochs=5, validation_data=(test_data, labels_test))

Epoch 1/5
65/65 [==============================] - 8s 101ms/step - loss: 0.7396 - acc: 0.6572 - val_loss: 1.2048 - val_acc: 0.2947
Epoch 2/5
65/65 [==============================] - 6s 96ms/step - loss: 0.2693 - acc: 0.8743 - val_loss: 2.1217 - val_acc: 0.2947
Epoch 3/5
65/65 [==============================] - 6s 98ms/step - loss: 0.0945 - acc: 0.9657 - val_loss: 0.5430 - val_acc: 0.7082
Epoch 4/5
65/65 [==============================] - 6s 94ms/step - loss: 0.0564 - acc: 0.9804 - val_loss: 0.5995 - val_acc: 0.6531
Epoch 5/5
65/65 [==============================] - 6s 94ms/step - loss: 0.0411 - acc: 0.9879 - val_loss: 0.5036 - val_acc: 0.7362


In [29]:
#predictions on test data
predicted=model.predict(test_data)
predicted

array([[0.13244271, 0.8675572 ],
       [0.7811859 , 0.2188141 ],
       [0.8710049 , 0.12899508],
       ...,
       [0.58285195, 0.41714808],
       [0.70398974, 0.29601032],
       [0.08317483, 0.9168251 ]], dtype=float32)

In [30]:
#model evaluation
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(labels_test,
predicted.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")
print(sklearn.metrics.classification_report(labels_test,
predicted.round()))

precision: [1.         0.52768166]
recall: [0.6260274 1.       ]
fscore: [0.77000842 0.69082673]
support: [730 305]
############################
              precision    recall  f1-score   support

           0       1.00      0.63      0.77       730
           1       0.53      1.00      0.69       305

   micro avg       0.74      0.74      0.74      1035
   macro avg       0.76      0.81      0.73      1035
weighted avg       0.86      0.74      0.75      1035
 samples avg       0.74      0.74      0.74      1035



RNN model

In [31]:
#import library
from keras.layers.recurrent import SimpleRNN

In [32]:
#model training
print('Training SIMPLERNN model.')
model = Sequential()
model.add(Embedding(MAX_NB_WORDS,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH))
model.add(SimpleRNN(2, input_shape=(None,1)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy']) 

Training SIMPLERNN model.


In [33]:
model.fit(train_data, labels_train, batch_size=16, epochs=5, validation_data=(test_data, labels_test))

Epoch 1/5
259/259 [==============================] - 18s 65ms/step - loss: 0.5360 - accuracy: 0.7887 - val_loss: 0.4191 - val_accuracy: 0.8589
Epoch 2/5
259/259 [==============================] - 17s 66ms/step - loss: 0.2771 - accuracy: 0.9645 - val_loss: 0.3035 - val_accuracy: 0.9024
Epoch 3/5
259/259 [==============================] - 18s 68ms/step - loss: 0.1525 - accuracy: 0.9901 - val_loss: 0.2650 - val_accuracy: 0.9014
Epoch 4/5
259/259 [==============================] - 18s 68ms/step - loss: 0.0981 - accuracy: 0.9966 - val_loss: 0.2472 - val_accuracy: 0.9063
Epoch 5/5
259/259 [==============================] - 17s 66ms/step - loss: 0.0690 - accuracy: 0.9985 - val_loss: 0.2379 - val_accuracy: 0.9092
